In [2]:
!pip install google-generativeai

In [10]:
import pandas as pd

file_path = "./prepro_company_data.csv"
df = pd.read_csv(file_path)
df["Positive_Score"] = None
df

,company,code,Date,Title,Positive_Score
0,삼성전자,'005930,2021-03-12,진짜 5 G 28 서비스 활성화 첫 발 과기부 TF 발족 회의,None
1,삼성전자,'005930,2021-03-05,아버님 댁 에어 프라이어 놓다 드리다,None
2,삼성전자,'005930,2021-05-14,ESG 경영 첫 단추 늘다 여성 관리자 키우다,None
3,삼성전자,'005930,2021-02-05,삼성 공장 증설 19조 투자 주 정부 9000억 세제 요청,None
4,삼성전자,'005930,2021-01-22,화웨이 서다 분리 중 가폰 어너 인텔 삼성 반도체 계약,None
...,...,...,...,...,...
15895,코오롱인더,'120110,2023-07-21,연구개발 1.1조 쏟다 코오롱 인 더 하반기 과실 딴다,None
15896,코오롱인더,'120110,2023-10-20,코오롱 인 더 3 Q 영업 익 시장 기 대치 회 4 Q 패션 성 수기 효과 기대 IBK,None
15897,코오롱인더,'120110,2023-10-27,코오롱 몰다 파격 할인 2023 OLO 모드 K 쇼핑 위 크다 진행,None
15898,코오롱인더,'120110,2023-12-01,추다 날씨 아우 터 판매 확대 코오롱 인 더 실적 개선 전망 IBK,None


In [12]:
import google.generativeai as genai
import os
import re
import time
import pandas as pd

# 파일 경로
file_path = "./gemini.csv"

# 파일 존재 여부 확인 및 데이터프레임 생성
if os.path.exists(file_path):
    df = pd.read_csv(file_path)
else:
    df = pd.DataFrame(columns=['Title', 'company', 'Positive_Score'])
    df.to_csv(file_path, index=False, encoding='utf-8-sig')

# Google API 키 설정
GOOGLE_API_KEY = "your_key"
genai.configure(api_key=GOOGLE_API_KEY)

# 보안 설정
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

# 시스템 명령 정의
system_instruction = "금융전문가로 가정해주세요. 당신은 주식 추천 경험이 있는 금융 전문가 입니다."

# 모델 초기화 (보안 설정 및 시스템 명령 포함)
model = genai.GenerativeModel('gemini-pro', safety_settings=safety_settings, system_instruction=system_instruction)
chat = model.start_chat(history=[])

# 감성 분석 및 점수 저장
for idx, row in df.iterrows():
    title = row['Title']
    retry_count = 0
    max_retries = 10
    company = row['company']

    if pd.notnull(row['Positive_Score']):
        continue

    while retry_count < max_retries:
        try:
            print(idx)
            prompt = f"헤드라인: {title}. 좋은 뉴스, 나쁜 뉴스, 불확실한 뉴스 세 가지가 될 확률이 합쳐서 1일 때 좋은 뉴스일 정도를 확률로만 답변해주세요. (예시답변: 0.2) {company}에 좋은 영향을 숫자로만 답해주세요"
            response = chat.send_message(prompt)
            score = re.findall(r'\d+\.\d+', response.text)
            if score:
                df.at[idx, 'Positive_Score'] = float(score[0])

            df.to_csv(file_path, index=False, encoding='utf-8-sig')
            break
        except Exception as e:
            print(f"Error occurred: {e}. Retrying in 5 seconds...")
            model = genai.GenerativeModel('gemini-pro', safety_settings=safety_settings, system_instruction=system_instruction)
            chat = model.start_chat(history=[])
            retry_count += 1
            time.sleep(10)
            df = pd.read_csv(file_path)

print("완료되었습니다.")


0
1
2
3
4
5
Error occurred: 429 Resource has been exhausted (e.g. check quota).. Retrying in 5 seconds...


KeyboardInterrupt: 